In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,LSTM
import numpy as np

batch_size=64
epochs=100
latent_dim=256
num_samples= 10000
data_path = '/content/sample_data/fra.txt'

In [3]:
input_texts=[]
target_texts=[]
input_characters = set()
target_characters = set()

with open(data_path,'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines)-1)]:
    input_text, target_text,_ = line.split('\t')
    target_text = '/t' + target_text + '/n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print('Number of samples', len(input_texts))
print('Number of unique input tokens', num_encoder_tokens)
print('Number of unique output tokens', num_decoder_tokens)
print('Max sequence length of inputs', max_encoder_seq_length)
print('Max sequence length of outputs', max_decoder_seq_length)

Number of samples 10000
Number of unique input tokens 71
Number of unique output tokens 91
Max sequence length of inputs 15
Max sequence length of outputs 61


In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([char, i] for i, char in enumerate(target_characters))

In [7]:
encoder_input_data= np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype= 'float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    #print(input_text)
    for t, char in enumerate(input_text):
        #print(t,char)
        encoder_input_data[i,t, input_token_index[char]] =1
    encoder_input_data[i, t+1:, input_token_index[' ']] =1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t, target_token_index[char]] =1
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]]= 1
    decoder_input_data[i,t+1:, target_token_index[' ']]= 1
    decoder_target_data[i,t:, target_token_index[' ']] = 1

In [9]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_output, state_h, state_c = encoder(encoder_inputs)
encoder_states=[state_h, state_c]

In [10]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state=True, return_sequences=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size= batch_size, epochs= epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 11s 19ms/step - loss: 1.1451 - accuracy: 0.7279 - val_loss: 1.0300 - val_accuracy: 0.7091
Epoch 2/100
125/125 [==============================] - 2s 12ms/step - loss: 0.7946 - accuracy: 0.7877 - val_loss: 0.7866 - val_accuracy: 0.7818
Epoch 3/100
125/125 [==============================] - 2s 13ms/step - loss: 0.6304 - accuracy: 0.8226 - val_loss: 0.6902 - val_accuracy: 0.7962
Epoch 4/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5611 - accuracy: 0.8387 - val_loss: 0.6214 - val_accuracy: 0.8175
Epoch 5/100
125/125 [==============================] - 2s 13ms/step - loss: 0.5156 - accuracy: 0.8487 - val_loss: 0.5836 - val_accuracy: 0.8308
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.4818 - accuracy: 0.8585 - val_loss: 0.5561 - val_accuracy: 0.8377
Epoch 7/100
125/125 [==============================] - 2s 12ms/step - loss: 0.4534 - accuracy: 0.8659 - val_loss: 0.5226 - val_accuracy

In [24]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape= (latent_dim,))
decoder_state_input_c = Input(shape= (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict( (i,char) for char ,i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1,1,num_decoder_tokens))
  target_seq[0,0,target_token_index[' ']] = 1.
  stop_condition = False
  decoded_sentence= ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq]+ states_value)
    sampled_token_index = np.argmax(output_tokens[0,-1,:])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    if(sampled_char =='\n' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    target_seq = np.zeros((1,1, num_decoder_tokens))
    target_seq[0,0, sampled_token_index] = 1

    states_value = [h,c]
  return decoded_sentence

for seq_index in range(100):
  input_seq = encoder_input_data[seq_index: seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence : ', input_texts[seq_index])
  print('Decoded sentence : ', decoded_sentence)

-
Input sentence :  Go.
Decoded sentence :  tMarche./n                                                    
-
Input sentence :  Go.
Decoded sentence :  tMarche./n                                                    
-
Input sentence :  Go.
Decoded sentence :  tMarche./n                                                    
-
Input sentence :  Hi.
Decoded sentence :  tSalut !/n                                                    
-
Input sentence :  Hi.
Decoded sentence :  tSalut !/n                                                    
-
Input sentence :  Run!
Decoded sentence :  tFilez !/n                                                    
-
Input sentence :  Run!
Decoded sentence :  tFilez !/n                                                    
-
Input sentence :  Run!
Decoded sentence :  tFilez !/n                                                    
-
Input sentence :  Run!
Decoded sentence :  tFilez !/n                                                    


KeyboardInterrupt: ignored

In [22]:
target_token_index

{' ': 0,
 '!': 1,
 '%': 2,
 '&': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '/': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '5': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '\xa0': 69,
 '«': 70,
 '»': 71,
 'À': 72,
 'Ç': 73,
 'É': 74,
 'Ê': 75,
 'à': 76,
 'â': 77,
 'ç': 78,
 'è': 79,
 'é': 80,
 'ê': 81,
 'î': 82,
 'ï': 83,
 'ô': 84,
 'ù': 85,
 'û': 86,
 'œ': 87,
 '\u2009': 88,
 '’': 89,
 '\u202f': 90}